In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from requests_html import HTML
from random import randint
from time import sleep

/home/aboubakar/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# function to extract html document from given url
def getHTMLdocument(url):
    
      
    # request for HTML document of given url
    response = requests.get(url)
      
    # response will be provided in JSON format
    return response.text
    
    
def get_url_and_context(start_page, end_page, tag): #start_page, end_page, tag):
    
    base_url = "https://stackoverflow.com/questions/tagged/"
    tag = tag 
    query_filter = "tab=votes&page="
    
    page_url = []
    contexte = []
    for page in range(start_page, end_page):
            
        
        url_to_scrape =  f"{base_url}{tag}?{query_filter}"+str(page)+str("&pagesize=50")
        prefix='https://stackoverflow.com'
        #print(url_to_scrape)
        # create document
        html_document = getHTMLdocument(url_to_scrape)
        # create soup object
        soup = BeautifulSoup(html_document, 'html.parser')

        #for page in range(start_page, end_page):
        for link in soup.find_all('a', class_="s-link", attrs={'href': re.compile("^/questions")}):
            # display the actual urls
            #print(prefix+link.get('href')) 
            page_url.append(prefix+link.get('href'))
        sleep(randint(2,10))
        
        r = requests.get(url_to_scrape)
        html_str = r.text
        html = HTML(html = html_str)

        question_summaries = html.find(".s-post-summary--content-excerpt")
        #print(question_summaries[5].text)
        
        i = 0
        for quest in question_summaries:
            contexte.append(question_summaries[i].text)
            i = i+1
        
            
    return page_url, contexte




In [3]:
def extract_question_context_answer(topic):
    url_to_scrape, context = get_url_and_context(1, 3, topic)
    question = []
    answer   = []

    for url in url_to_scrape:
        r = requests.get(url)
        html_str = r.text
        html = HTML(html = html_str)
        #print(url)
        question_summary= html.find(".question-hyperlink")
        question.append(question_summary[0].text)
        answer_summary  = html.find(".comment-copy")
        answer.append(answer_summary[0].text)
        #print(question_summary[0].text)
        #print("\n")
        #print(answer_summary[0].text)
        
    
    return question, context, answer

In [4]:
Q1, C1, A1 = extract_question_context_answer('python')


In [5]:
Q2, C2, A2 = extract_question_context_answer('pandas')

In [6]:
Q3, C3, A3 = extract_question_context_answer('php')

In [7]:
Q4, C4, A4 = extract_question_context_answer('html')

In [8]:
Q5, C5, A5 = extract_question_context_answer('bash')

In [9]:
Q6, C6, A6 = extract_question_context_answer('matlab')

In [10]:
Q7, C7, A7 = extract_question_context_answer('TypeScript')

In [15]:
Q8, C8, A8 = extract_question_context_answer('json') 

In [12]:
Q9, C9, A9 = extract_question_context_answer('android')

In [13]:
Q10, C10, A10 = extract_question_context_answer('Kotlin') 

In [16]:
Q11, C11, A11 = extract_question_context_answer('array')

In [17]:
df_Q = Q1 +Q2 +Q3 +Q4 +Q5 +Q6 +Q7 + Q8+Q9 +Q10 +Q11 
df_C = C1 +C2 +C3 +C4 +C5 +C6 +C7 + C8+ C9 +C10 +C11 
df_A = A1 +A2 +A3 +A4 +A5 +A6 +A7 + A8 +A9 +A10 +A11

In [18]:
df=pd.DataFrame()
df["question"]= df_Q
df["Summary"]= df_C
df["answer"]  = df_A

In [19]:
df.shape[0]

899

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  899 non-null    object
 1   Summary   899 non-null    object
 2   answer    899 non-null    object
dtypes: object(3)
memory usage: 21.2+ KB


In [21]:
df.head()

,question,Summary,answer
0,"What does the ""yield"" keyword do?",What is the use of the yield keyword in Python...,It's not only about the technical finesse on h...
1,"What does if __name__ == ""__main__"": do?","What does this do? if __name__ == ""__main__"": ...","Just for the record - what is ""main"": docs.pyt..."
2,Does Python have a ternary conditional operator?,If Python doesn't have a ternary conditional o...,In the Python 3.0 official documentation refer...
3,What are metaclasses in Python?,"In Python, what are metaclasses and what do we...",It appears that in Django models.Model it does...
4,How do I check whether a file exists without e...,"How do I check whether a file exists or not, w...",To check whether a Path object exists independ...


In [22]:
df.to_csv("df_question_summary_answer_stackoverflow.csv")